## Import Data and Libraries

In [1]:
from pyspark import SparkContext
from spark_sklearn import GridSearchCV

from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.preprocessing import MinMaxScaler 
from time import time
from math import sqrt
from sklearn.linear_model import ElasticNet, SGDRegressor, LinearRegression, ElasticNetCV, LassoCV, Lasso, Ridge, RidgeCV, BayesianRidge, ARDRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor

from matplotlib import pyplot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
%matplotlib inline

data = pd.read_csv('final.csv')
data = data.drop(columns=['date','low','open','high','marketcap'])
#data = data[data.columns[sel]]
print('n_features:', len(data.iloc[0]))
print('n_samples:', len(data))
data.tail()

n_features: 36
n_samples: 2735


,close,volatility,volume,google_trends,gold,silver,platinum,palladium,oil,usd_eur,...,TXN_per_block,est_TXN_vol,cost_per_TXN,total_TXN_fees,usd_trade_vol,hash_rate,avg_block_size,difficulty,num_unique_addr,miners_revenue
2730,6093.67,0.031736,3.279760e+09,6.0,1260.30,16.225,864.0,939.0,75.23,1.1672,...,1274.987261,675604827.0,61.446716,213583.5164,501623219.9,39627403.83,0.920948,5.080000e+12,415602,1.209255e+07
2731,6157.13,0.020789,3.296220e+09,6.0,1254.60,16.210,860.0,946.0,77.41,1.1616,...,1338.383562,609799528.3,59.349571,201953.0378,363193880.4,36850961.53,1.067592,5.080000e+12,410397,1.139519e+07
2732,5903.44,0.049391,3.467800e+09,6.0,1251.55,16.110,852.0,945.0,73.45,1.1583,...,1557.933884,795615808.3,50.289212,197368.2529,345979167.6,30540865.37,1.088043,5.080000e+12,397865,9.288788e+06
2733,6218.30,0.070443,3.966230e+09,8.0,1250.45,16.030,851.0,953.0,74.13,1.1658,...,1195.057325,703945479.4,65.127555,166798.5735,262900494.4,39627403.83,0.689430,5.080000e+12,396405,1.205269e+07
2734,6404.00,0.039642,4.543860e+09,7.0,1250.45,16.030,851.0,953.0,74.13,1.1658,...,1310.470199,656285943.9,57.609894,163181.8222,414797814.1,38112980.76,0.746513,5.080000e+12,453050,1.123672e+07


In [2]:
test_data = pd.read_csv('Test_Data.csv')
test_data = test_data.drop(columns=['date','low','open','high','marketcap'])
print('n_features:', len(test_data.iloc[0]))
print('n_samples:', len(test_data))
test_data.head()

n_features: 36
n_samples: 92


,close,volatility,volume,google_trends,gold,silver,platinum,palladium,oil,usd_eur,...,TXN_per_block,est_TXN_vol,cost_per_TXN,total_TXN_fees,usd_trade_vol,hash_rate,avg_block_size,difficulty,num_unique_addr,miners_revenue
0,6385.82,0.022569,4788259840,62,1250.45,16.030,851.0,953,74.13,1.1658,...,1284.148936,493208885.1,58.735614,140545.4062,489427084.1,35588942.30,0.987340,5.077500e+12,368307,10494418.50
1,6614.18,0.058242,4396930048,72,1247.80,15.980,839.0,941,73.89,1.1639,...,1062.904762,991949405.1,75.393733,106682.7087,224327884.6,37103365.37,0.705291,5.077500e+12,341861,11673361.88
2,6529.59,0.034094,4672309760,69,1251.75,15.930,838.0,954,74.19,1.1665,...,1413.432624,720058629.1,57.309569,150160.0866,281178824.1,37594816.26,0.987550,5.172890e+12,433257,11271293.25
3,6597.55,0.048633,4176689920,65,1255.65,16.045,834.0,948,74.19,1.1665,...,1868.508621,872392592.0,45.224241,185064.5374,356469308.6,30929068.69,1.113817,5.363680e+12,456149,9617154.00
4,6639.14,0.030521,4999240192,69,1255.50,15.950,845.5,947,73.05,1.1709,...,1543.194030,665699858.5,54.318173,142640.2951,345023639.3,35728406.94,0.935353,5.363680e+12,435401,11089706.00


In [3]:
# Combined train and test sets
combined_data = pd.concat([data[2613:], test_data], ignore_index=True)
combined_data['Price'] = combined_data['close'].shift(-1)
print('n_features:', len(combined_data.iloc[0]))
print('n_samples:', len(combined_data))
combined_data = combined_data.iloc[:-1,1:]
combined_data.tail()

n_features: 37
n_samples: 214


,volatility,volume,google_trends,gold,silver,platinum,palladium,oil,usd_eur,usd_jpy,...,est_TXN_vol,cost_per_TXN,total_TXN_fees,usd_trade_vol,hash_rate,avg_block_size,difficulty,num_unique_addr,miners_revenue,Price
208,0.034161,4.726180e+09,61.0,1201.90,14.290,827.0,1065.0,73.40,1.1777,0.00905,...,503494967.4,52.112921,102965.2698,273203917.3,52622781.21,0.807114,7.152630e+12,469028,12385677.98,6495.00
209,0.028964,4.437300e+09,59.0,1194.25,14.475,824.0,1059.0,72.22,1.1737,0.00904,...,842041019.0,54.247914,130284.3136,390313648.8,60089527.19,0.768241,7.152630e+12,490588,13676604.30,6676.75
210,0.037517,4.606810e+09,58.0,1185.40,14.420,812.0,1067.0,72.18,1.1707,0.00905,...,748444892.4,46.610199,130238.5658,312748554.4,49422747.22,0.898005,7.152630e+12,474079,11369486.09,6644.13
211,0.027904,5.014430e+09,58.0,1187.25,14.305,815.0,1094.0,73.16,1.1576,0.00906,...,824319235.8,47.597177,154586.9390,283381555.4,50844984.55,0.877031,7.152630e+12,481907,11836751.48,6601.96
212,0.019986,4.363690e+09,48.0,1187.25,14.305,815.0,1094.0,73.16,1.1576,0.00902,...,913410388.9,51.317781,193097.0923,457654461.3,54756137.20,0.855169,7.152630e+12,527057,13046981.40,6625.56


In [4]:
# Data normalization
def normalize_data(X,Y=None):
    scaler = MinMaxScaler()
    return scaler.fit_transform(X,Y)

# Define time-series cross validation split
def TimeSeriesCVSplit(n_samples, test_size, step_size):
    data_split = []
    train_end = n_samples - test_size - step_size 
    test_start = train_end + step_size
    for i in range(0,test_size):
        train_index = list(range(0, train_end))
        test_index = [test_start]
        data_split.append([train_index,test_index])
        train_end+=1
        test_start+=1
    return data_split

def wfss(X,Y,model,subset):
    sel = subset.copy() # # Selected features
    overall_error = train_model(X[:,sel],Y,model,predict=False) 
    while len(sel) != 0:
        # Select candidate
        cand_error = 1e10 # Assign a big number
        for cand in sel:
            features = sel.copy()
            features.remove(cand)
            if len(features) > 1:
                new_error = train_model(X[:,features],Y,model,predict=False)
            else:
                new_error = train_model(X[:,features[0]].reshape(-1,1),Y,model,predict=False)
            if new_error < cand_error:
                selected_candidate = cand
                cand_error = new_error
        if overall_error < cand_error:
            # Stop if the new candidate doesn’t
            # improve the assessment of the
            # previously selected candidates
            break
        else:
            overall_error = cand_error
            sel.remove(selected_candidate)
    #print('RMSE:', str(train_model(X[:,sel],Y,model,predict=False)))
    return sel


def train_model(X,Y,model,predict):
    data_split = TimeSeriesCVSplit(len(X),n_validation,0)
    Y_pred = []
    Y_test = []
    scores = []
    n=1
    for fold in data_split:
        X_train = X[fold[0]]
        Y_train = Y[fold[0]]
        X_test = X[fold[1]]
        y_pred = Y[fold[1]]
        
        import timeit
        start = timeit.default_timer()
        
        if predict:
            #print(n)
            #Fine-tune alpha
            #k, p, weights = fine_tune_KNN(X_train,Y_train)
                # Feature Selection
            selected_features = wfss(X_train,Y_train,model,np.arange(0,35).tolist())         
            print(n, selected_features)
            
            scaler = MinMaxScaler()
            scaler.fit(X_train[:,selected_features],Y_train)
            X_train = scaler.transform(X_train[:,selected_features])
            X_test = scaler.transform(X_test[:,selected_features])  
            
        if not predict:
            scaler = MinMaxScaler()
            scaler.fit(X_train,Y_train)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test) 
        
        
        model.fit(X_train,Y_train.reshape(-1,))
        prediction = model.predict(X_test).reshape(-1,1)
        Y_pred.append(prediction)
        Y_test.append(y_pred)
        rmse = np.sqrt(mean_squared_error(y_pred, prediction))
        scores.append(rmse)
        
        if predict:
            stop = timeit.default_timer()
            print('Time: {:0.2f} mins'.format((stop - start)/60))
        n+=1
    Y_pred = np.asarray(Y_pred)
    Y_test = np.asarray(Y_test)
    if predict:
        return np.sqrt(mean_squared_error(Y_pred.reshape(-1,1), Y_test.reshape(-1,1))), Y_test, Y_pred
    if not predict:
        return np.sqrt(mean_squared_error(Y_pred.reshape(-1,1), Y_test.reshape(-1,1)))


def get_feature_ranking(feature_scores):
    indices = np.nonzero(feature_scores)[0].tolist()
    scores = np.abs(feature_scores[indices]).tolist() 
    sorted_scores, sorted_features = zip(*sorted(zip(scores, indices),reverse=True))
    return list(sorted_features)

def RMSE(y_true, y_hat):
    error = np.sqrt(mean_squared_error(y_true, y_hat))
    return error

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


## Data Preparation

In [5]:
# Feature Subsets
lasso_subset = [34, 28, 30, 32, 1, 27, 2, 8, 25, 12, 22, 0, 17, 6, 18, 24, 9, 5, 31, 10, 19, 33, 23, 14] # lasso [0:20] for KNN

### TO BE UPDATED ###
etrees_subset = [34, 32, 14, 18, 27, 22, 1, 6, 23, 30, 16, 15, 19, 29, 28, 13, 26, 7, 8, 11, 2, 33, 3] # extra trees
#subset4 = [0, 1, 24, 2, 31, 3, 29, 27, 34, 28, 7, 25, 26, 33, 4, 8, 6, 11, 17, 21, 30, 23, 22] # xgboost can remove 9
xgb_subset = [0, 1, 29, 31, 27, 6, 28, 24, 25, 34, 7, 2, 8, 3, 26, 22, 4, 11, 21, 33, 20, 30, 17] # xgboost can remove 9
subsets = [lasso_subset,etrees_subset,xgb_subset]

# Validation samples size (1/4/18 - 30/6/18)
n_validation = 91

data = combined_data.values # use best period

n_samples = len(data)
Y = data[:-92,-1].reshape(-1,1)
X = data[:-92,:-1]



### Parameters Tuning

In [6]:
from sklearn.pipeline import Pipeline
pipe = pipe = Pipeline([
    ('normalize', MinMaxScaler()),
    ('extrees', ExtraTreesRegressor())
])
pipe

Pipeline(memory=None,
     steps=[('normalize', MinMaxScaler(copy=True, feature_range=(0, 1))), ('extrees', ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
          oob_score=False, random_state=None, verbose=0, warm_start=False))])

In [7]:
%%time
sc = SparkContext()

param_grid = [{"extrees__max_features": [10,15,18,20,22,25,30],
              "extrees__n_estimators": [40,50,60],
              "extrees__random_state": [1,100],
              "extrees__max_depth": [10,None],
              "extrees__n_jobs": [-1],
              "extrees__bootstrap": [True,False]
        }]
split = TimeSeriesCVSplit(len(X),n_validation,0)
gs = GridSearchCV(sc, pipe, param_grid, cv=split, scoring=make_scorer(RMSE,greater_is_better=False), verbose=1) 
gs.fit(X,Y.reshape(-1,))
print("Best RMSE:", str(gs.cv_results_['mean_test_score'][gs.best_index_]))
print('Best Parameters: ',gs.cv_results_['params'][gs.best_index_])
sc.stop()

Fitting 91 folds for each of 168 candidates, totalling 15288 fits


KeyboardInterrupt: 

### Prediction

#### Replace the best parameters from above into the ExtraTreesRegressor() below

In [9]:
%%time
# Test Set
n_validation = 90
Y = data[:,-1].reshape(-1,1)
X = data[:,:-1]
rmse,Y_test,Y_pred= train_model(X,Y,ExtraTreesRegressor(),predict=True)
np.savetxt('ETrees.csv', Y_pred.reshape(-1,1), delimiter=',',  fmt='%1.10e')
print('Test RMSE:',str(rmse))


Train RMSE: 701.0224395604396
Test RMSE: 450.5154666666666
CPU times: user 2.05 s, sys: 13.5 ms, total: 2.07 s
Wall time: 2.14 s


### Training
#### Replace the best parameters from above into the ExtraTreesRegressor() below

In [ ]:
'''
%%time
# Train Set
n_validation = 91
Y = data[:-92,-1].reshape(-1,1)
X = data[:-92,:-1]

# Feature Selection
selected_features = wfss(X,Y,ExtraTreesRegressor(),np.arange(0,35).tolist())
print(selected_features)
rmse,Y_train_test,Y_train_pred= train_model(X[:,selected_features],Y,ExtraTreesRegressor(),predict=True)
np.savetxt('ETrees_train.csv', Y_train_pred.reshape(-1,1), delimiter=',',  fmt='%1.10e')
print('Train RMSE:',str(rmse))
'''

### VISUAlIZATION

#### True plot

In [52]:
trace1 = go.Scatter(
    x = np.arange(0, len(Y_pred), 1),
    y = Y_pred.reshape(-1,),
    mode = 'lines',
    name = 'Predicted labels',
    line = dict(color=('rgb(244, 146, 65)'), width=2)
)
trace2 = go.Scatter(
    x = np.arange(0, len(Y_test), 1),
    y = Y_test.reshape(-1,),
    mode = 'lines',
    name = 'True labels',
    line = dict(color=('rgb(66, 244, 155)'), width=2)
)

layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our model predicted',
             xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=[trace1, trace2], layout=layout)
py.iplot(fig, filename='results_demonstrating0')

print(np.sqrt(mean_squared_error(Y_test.reshape(-1,), Y_pred.reshape(-1,))))


338.0632205349688


#### Plot shifted -1

In [19]:

y1 = Y_pred.reshape(-1,)[2:]
y2 = Y_test.reshape(-1,)[0:-2]
trace1 = go.Scatter(
    x = np.arange(0, len(Y_pred), 1),
    y = y1,
    mode = 'lines',
    name = 'Predicted labels',
    line = dict(color=('rgb(244, 146, 65)'), width=2)
)
trace2 = go.Scatter(
    x = np.arange(0, len(Y_test), 1),
    y = y2,
    mode = 'lines',
    name = 'True labels',
    line = dict(color=('rgb(66, 244, 155)'), width=2)
)

layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our model predicted',
             xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=[trace1, trace2], layout=layout)
py.iplot(fig, filename='results_demonstrating0')
